# সেম্যান্টিক কোর 

এই কোড নমুনায়, আপনি [সেম্যান্টিক কোর](https://aka.ms/ai-agents-beginners/semantic-kernel) AI ফ্রেমওয়ার্ক ব্যবহার করে একটি সাধারণ এজেন্ট তৈরি করবেন।

এই নমুনার লক্ষ্য হল আপনাকে সেই ধাপগুলো দেখানো যা আমরা পরে বিভিন্ন এজেন্টিক প্যাটার্ন বাস্তবায়নের সময় অতিরিক্ত কোড নমুনায় ব্যবহার করব।


## প্রয়োজনীয় পাইথন প্যাকেজগুলি ইমপোর্ট করুন


In [ ]:
import json
import os 

from typing import Annotated

from dotenv import load_dotenv

from IPython.display import display, HTML

from openai import AsyncOpenAI

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent
from semantic_kernel.functions import kernel_function

## ক্লায়েন্ট তৈরি করা

এই উদাহরণে, আমরা [GitHub Models](https://aka.ms/ai-agents-beginners/github-models) ব্যবহার করব LLM-এ অ্যাক্সেস পাওয়ার জন্য।

`ai_model_id` কে `gpt-4o-mini` হিসেবে সংজ্ঞায়িত করা হয়েছে। GitHub Models মার্কেটপ্লেসে উপলব্ধ অন্য কোনো মডেলে পরিবর্তন করে দেখুন বিভিন্ন ফলাফল কেমন হয়।

`Azure Inference SDK` ব্যবহার করার জন্য, যা GitHub Models-এর `base_url` এর জন্য ব্যবহৃত হয়, আমরা Semantic Kernel-এর মধ্যে `OpenAIChatCompletion` সংযোগকারী ব্যবহার করব। এছাড়াও, Semantic Kernel ব্যবহার করে অন্যান্য মডেল প্রদানকারীদের জন্য [উপলব্ধ অন্যান্য সংযোগকারী](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion) রয়েছে।


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## এজেন্ট তৈরি করা

নিচে আমরা `TravelAgent` নামে এজেন্ট তৈরি করছি।

এই উদাহরণে, আমরা খুব সাধারণ নির্দেশাবলী ব্যবহার করছি। আপনি এই নির্দেশাবলী পরিবর্তন করতে পারেন এবং দেখতে পারেন এজেন্ট কীভাবে ভিন্নভাবে প্রতিক্রিয়া জানায়।


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## এজেন্ট চালানো

এখন আমরা `ChatHistory` সংজ্ঞায়িত করে এবং এতে `system_message` যোগ করে এজেন্ট চালাতে পারি। আমরা পূর্বে সংজ্ঞায়িত `AGENT_INSTRUCTIONS` ব্যবহার করব।

এগুলো সংজ্ঞায়িত করার পর, আমরা একটি `user_inputs` তৈরি করি যা ব্যবহারকারী এজেন্টকে পাঠাচ্ছে। এই ক্ষেত্রে, আমরা এই বার্তাটি `Plan me a sunny vacation` সেট করেছি।

এই বার্তাটি পরিবর্তন করতে পারেন এবং দেখুন এজেন্ট কীভাবে ভিন্নভাবে প্রতিক্রিয়া জানায়।


In [ ]:
user_inputs = [
    "Plan me a day trip.",
    "I don't like that destination. Plan me another vacation.",
]

async def main():
    thread: ChatHistoryAgentThread | None = None

    for user_input in user_inputs:
        html_output = (
            f"<div style='margin-bottom:10px'>"
            f"<div style='font-weight:bold'>User:</div>"
            f"<div style='margin-left:20px'>{user_input}</div></div>"
        )

        agent_name = None
        full_response: list[str] = []
        function_calls: list[str] = []

        # Buffer to reconstruct streaming function call
        current_function_name = None
        argument_buffer = ""

        async for response in agent.invoke_stream(
            messages=user_input,
            thread=thread,
        ):
            thread = response.thread
            agent_name = response.name
            content_items = list(response.items)

            for item in content_items:
                if isinstance(item, FunctionCallContent):
                    if item.function_name:
                        current_function_name = item.function_name

                    # Accumulate arguments (streamed in chunks)
                    if isinstance(item.arguments, str):
                        argument_buffer += item.arguments
                elif isinstance(item, FunctionResultContent):
                    # Finalize any pending function call before showing result
                    if current_function_name:
                        formatted_args = argument_buffer.strip()
                        try:
                            parsed_args = json.loads(formatted_args)
                            formatted_args = json.dumps(parsed_args)
                        except Exception:
                            pass  # leave as raw string

                        function_calls.append(f"Calling function: {current_function_name}({formatted_args})")
                        current_function_name = None
                        argument_buffer = ""

                    function_calls.append(f"\nFunction Result:\n\n{item.result}")
                elif isinstance(item, StreamingTextContent) and item.text:
                    full_response.append(item.text)

        if function_calls:
            html_output += (
                "<div style='margin-bottom:10px'>"
                "<details>"
                "<summary style='cursor:pointer; font-weight:bold; color:#0066cc;'>Function Calls (click to expand)</summary>"
                "<div style='margin:10px; padding:10px; background-color:#f8f8f8; "
                "border:1px solid #ddd; border-radius:4px; white-space:pre-wrap; font-size:14px; color:#333;'>"
                f"{chr(10).join(function_calls)}"
                "</div></details></div>"
            )

        html_output += (
            "<div style='margin-bottom:20px'>"
            f"<div style='font-weight:bold'>{agent_name or 'Assistant'}:</div>"
            f"<div style='margin-left:20px; white-space:pre-wrap'>{''.join(full_response)}</div></div><hr>"
        )

        display(HTML(html_output))

await main()


---

**অস্বীকৃতি**:  
এই নথিটি AI অনুবাদ পরিষেবা [Co-op Translator](https://github.com/Azure/co-op-translator) ব্যবহার করে অনুবাদ করা হয়েছে। আমরা যথাসম্ভব সঠিক অনুবাদের চেষ্টা করি, তবে অনুগ্রহ করে মনে রাখবেন যে স্বয়ংক্রিয় অনুবাদে ত্রুটি বা অসঙ্গতি থাকতে পারে। নথিটির মূল ভাষায় লেখা সংস্করণটিকেই প্রামাণিক উৎস হিসেবে বিবেচনা করা উচিত। গুরুত্বপূর্ণ তথ্যের জন্য, পেশাদার মানব অনুবাদ ব্যবহার করার পরামর্শ দেওয়া হয়। এই অনুবাদ ব্যবহারের ফলে সৃষ্ট কোনো ভুল বোঝাবুঝি বা ভুল ব্যাখ্যার জন্য আমরা দায়ী নই।
